# Homework 1 (Exploratory Data Analysis and Linear Regression)



*   Due Date: 13-10-2021, before the following lesson is proceeded.
*   Format: in IPython Notebook form, submit in Moodle submission module, name the file as follow: **HW1_(Your name)_(Student ID)**.
*   Succinct Reasoning, Codes and Output Presentation are prerequisite, and the marks would be allocated in grades. 
*   Make sure the layout is **nice and tidy** for my and your own sake.




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import missingno as msn
from collections import Counter

# Question 1: Exploratory Data Analysis (TMDB 5000 Movie Dataset)

https://www.kaggle.com/tmdb/tmdb-movie-metadata

1.   Calculate the number of missing values for each variable and show the one with the most missing values. 
2.   Define a new variable `earn` as `revenue` - `budget`, show the top 10 movies with the most `earn`. 
3.   Plot a Histogram for `vote_average`, bins = 50
4.   Plot a Bar Plot  for `original language`.
5.   Append the codes below to the one in (4) and compile.  **Briefly explain the function of the codes given**.
```
plt.xticks(size=12, rotation=80)`
plt.yscale(value = "log")
```
6.   Plot a Scatter Plot for `revenue` and `vote_average`.  **Briefly describe the trend observed**.
7.   Calculate the Correlation Matrix for `budget`, `revenue`, `vote_count`, `vote_average`.  **Briefly describe the trend observed**.
8.   Plot a Box Plot for `release_year` with 10-year interval. **Briefly describe the trend observed**.

In [ ]:
df = pd.read_csv('hw1_archive/tmdb_5000_movies.csv')

In [ ]:
for col_name, count_nan in zip(df.columns, df.isna().sum()):
    if count_nan == max(df.isna().sum()): print(col_name, count_nan)

In [ ]:
if 'earn' not in df.columns: df.insert(0, 'earn', df.revenue-df.budget)
df.sort_values(["earn"],ascending=False,inplace=True)
df.iloc[0:10,[7,0]]

In [ ]:
sns.histplot(df.vote_average, bins=50)

In [ ]:
fig = plt.figure(figsize=(8,5), dpi = 100)
ax = sns.distplot(df["vote_average"], bins = 50, kde = False)

In [ ]:
plt.figure(figsize=(30, 15))
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
sns.countplot(x="original_language", data=df)

In [ ]:
plt.figure(figsize=(15,5))
plt.xticks(rotation=80)
plt.yscale(value='log')
sns.countplot(x="original_language", data=df)

**note**: 用`log`來壓縮在資料中特別大的值成次方，這樣越大的值就會被壓縮得越多

In [ ]:
plt.figure(figsize=(10,5), dpi=100)
sns.scatterplot(x="revenue", y="vote_average", data=df)

In [ ]:
plt.figure(figsize = (10,6), dpi = 100)
ax = sns.regplot(x = "revenue", y = "vote_average", data = df, 
                 scatter_kws = {"alpha":0.2})

**note**: 多數的`vote_average`落在4-8之間，`vote_average`似乎並不和`revenue`成正比 / 正相關

In [ ]:
sns.heatmap(df[['budget', 'revenue', 'vote_count', 'vote_average']].corr(), annot = True)

**note**: `vote_average`和`budget`有最低的相關性，似乎可以解釋說`budget`並不會影響`vote_average`。`vote_count`和`revenue`有著最高的相關性，也許可以說`vote_count`會連帶影響`revenue`

In [ ]:
df['release_year'] = pd.DatetimeIndex(df.release_date).year
df['release_year'] = pd.cut(df['release_year'], bins=[year for year in range(1910, 2021, 10)])
plt.figure(figsize=(8, 5), dpi=100)
plt.xticks(size=8, rotation=50)
sns.boxplot(x='release_year', y='vote_average', data=df)

In [ ]:
[date
 for date in df.release_date.values 
 if type(date) == str and int(date[:4]) <= 1920]

**note**: 

1910-1920中只有一筆資料所以不考慮。

1940-1950的`vote_average`最集中，並且在這之後有逐年越來越離散的情況

# Question 2: Linear Regression (Airline Passenger Satisfaction)

https://www.kaggle.com/teejmahal20/airline-passenger-satisfaction

There're 2 csv files within. `train.csv` is for fitting the model while `test.csv` is for validating.
1.   Missing Data Handling - Visualize the Missing Values with `missingno`.  Show the variable(s) with Missing Values and its(their) respective number.
2.   Visualization (Correlation Heatmap) - Compile the codes given below and **briefly discribe the trend observed**.
```
x = df.loc[:,'Flight Distance':'Cleanliness']
plt.figure(figsize=(20,10))
c= x.corr()
sns.heatmap(c)
```
3.   Rename all the variables with the replacement of space( ) to underscore(_).
4.   Convert the value in `satisfaction` as 
`{'neutral or dissatisfied':0, 'satisfied':1}`
5.   Conduct a Linear Regression Model with 
```
x = df.loc[:,'Flight Distance':'Cleanliness']
y = df['satisfaction']
```
6.   Evaluate your model with R-square and MSE.  Validate with 'test.csv' 
7.   Conduct a Poynomial Regression with `patsy` design matrix. Evaluate with R-square and MSE.  Validate with 'test.csv'.

In [ ]:
train = pd.read_csv('hw1_archive/train.csv')
train.set_index('Index', inplace=True)
test = pd.read_csv('hw1_archive/test.csv')
test.set_index('Index', inplace=True)

In [ ]:
msn.matrix(train)

In [ ]:
from typing import TypeVar

DataFrame = TypeVar('pd.core.frame.DataFrame') # for better typing hint
def show_nan_details(dataframe: DataFrame):
    '''showing which column has the most missing value and thier index'''
    
    print(f'variable(s) with Missing Values and its(their) respective number:\n')
    for col_name, count_nan in zip(dataframe.columns, dataframe.isna().sum()):
        if count_nan == max(dataframe.isna().sum()): 
            print(f'there are {count_nan} rows are missing in \'{col_name}\'\n')
    print(f"and their index:\n{list(dataframe.loc[dataframe['Arrival Delay in Minutes'].isna()].index)}")

In [ ]:
show_nan_details(train)

In [ ]:
msn.matrix(test)

In [ ]:
show_nan_details(test)

In [ ]:
x = train.loc[:,'Flight Distance':'Cleanliness']
plt.figure(figsize=(20,10))
c= x.corr()
sns.heatmap(c, annot=True)

In [ ]:
def transform_col_name(dataframe: DataFrame) -> dict:
    transform_dict = {}
    for col in dataframe.columns:
        if ' ' in col:
            transform_dict[col] = col.replace(' ', '_')
    return transform_dict

In [ ]:
train.rename(columns = transform_col_name(train), inplace = True)
train.columns

In [ ]:
df.columns = df.columns.str.replace(' ', '_')

In [ ]:
dummy_satisfaction = lambda dataframe: [0 if value == 'neutral or dissatisfied' else 1 
                                        for value in dataframe.satisfaction.values]
assert len(dummy_satisfaction(train)) == len(train.satisfaction)

In [ ]:
train['satisfaction'] = dummy_satisfaction(train)
#train.head(3)

In [ ]:
# another way
# df['satisfaction'] = df['satisfaction'].replace({'neutral or dissatisfied':0,'satisfied':1})

In [ ]:
test.rename(columns = transform_col_name(test), inplace = True)
test.columns

In [ ]:
test['satisfaction'] = dummy_satisfaction(test)
#test.head(3)

In [ ]:
x_train = train.loc[:,'Flight_Distance':'Cleanliness']
y_train = train['satisfaction']

In [ ]:
len(x_train), len(y_train)

In [ ]:
x_test = test.loc[:,'Flight_Distance':'Cleanliness']
y_test = test['satisfaction']

In [ ]:
from sklearn.preprocessing import MinMaxScaler # (X - Xmin) / Range
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x_test)
x_test = scaler.transform(x_test)

In [ ]:
from sklearn import linear_model
lm = linear_model.LinearRegression(fit_intercept = True, normalize = False)

In [ ]:
lm_fitted = lm.fit(x_train, y_train)

In [ ]:
y_hat_train = lm_fitted.predict(x_train)
y_hat_test = lm_fitted.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
print('Performance on Training Data')
print('  + mean squared error: %.2f' % mean_squared_error(y_train, y_hat_train))
print('  + coefficient of determination: %.2f' % r2_score(y_train, y_hat_train))
print('\n')
print('Performance on Testing Data')
print('  + mean squared error: %.2f' % mean_squared_error(y_test, y_hat_test))
print('  + coefficient of determination: %.2f' % r2_score(y_test, y_hat_test))

Conduct a `Poynomial Regression` with `patsy` `design matrix`. Evaluate with `R-square` and `MSE`. Validate with 'test.csv'.

In [ ]:
from patsy import dmatrix
formula = "(" + " + ".join(list(train.drop(columns = ['id', 'Gender', 'Customer_Type', 
                                                      'Age', 'Type_of_Travel', 'Class', 
                                                      'satisfaction']).columns)) + ")" + "**2"

In [ ]:
formula

In [ ]:
x_poly = dmatrix(formula, data = train.drop(columns = ['id', 'Gender', 'Customer_Type', 
                                                    'Age', 'Type_of_Travel', 'Class', 
                                                    'satisfaction']), return_type = "dataframe")

In [ ]:
y_poly = dmatrix(formula, data = test.drop(columns = ['id', 'Gender', 'Customer_Type', 
                                                    'Age', 'Type_of_Travel', 'Class', 
                                                    'satisfaction']), return_type = "dataframe")

In [ ]:
len(x_poly), len(y_poly)

In [ ]:
x_poly.head(3)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x_poly)
x_train = scaler.transform(x_poly)

In [ ]:
len(x_train), len(y_train)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(y_poly)
x_test = scaler.transform(y_poly)

In [ ]:
lm = linear_model.LinearRegression(fit_intercept = True, normalize = False)

In [ ]:
lm_fitted = lm.fit(x_train, y_train[:len(x_train)])

In [ ]:
y_hat_train = lm_fitted.predict(x_train)
y_hat_test = lm_fitted.predict(x_test)

In [ ]:
len(y_hat_train), len(y_hat_test)

In [ ]:
print('Performance on Training Data')
print('  + mean squared error: %.2f' % mean_squared_error(y_train[:len(x_train)], y_hat_train))
print('  + coefficient of determination: %.2f' % r2_score(y_train[:len(x_train)], y_hat_train))
print('Performance on Testing Data')
print('  + mean squared error: %.2f' % mean_squared_error(y_test[:len(y_hat_test)], y_hat_test))
print('  + coefficient of determination: %.2f' % r2_score(y_test[:len(y_hat_test)], y_hat_test))

# Bonus (Visualize with Altair)

Plot the graphs in Question 1 with Altair.  Try your best will do.

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
df = pd.read_csv("hw1_archive/tmdb_5000_movies.csv")

In [ ]:
alt.Chart(df).mark_bar().encode(
    alt.X('vote_average:Q', bin=alt.BinParams(maxbins=50)),
    alt.Y('count()')
).properties(
    width = 500,
    height = 300
)

In [ ]:
alt.Chart(df).mark_bar().encode(
    alt.X('original_language:N'),
    alt.Y('count()')
).properties(
    width = 1000,
    height = 300
)

In [ ]:
alt.Chart(df).mark_point().encode(
    x='revenue:Q',
    y='vote_average:Q'
).properties(
    width = 500,
    height = 300
)

In [ ]:
df["release_year"] = pd.DatetimeIndex(df["release_date"]).year
df["release_year"] = pd.cut(df["release_year"], bins = [1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010,2020])

In [ ]:
df["release_year"] = df["release_year"].astype(str)

In [ ]:
alt.Chart(df).mark_boxplot().encode(
    alt.X('release_year:N'),
    alt.Y('vote_average:Q')
).properties(
    width = 500,
    height = 300
)

In [ ]:
!ls -lh